In [7]:
# Libraries
import pandas as pd  
import numpy as np  
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV 
from sklearn.linear_model import Perceptron, LogisticRegressionCV, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier, Lasso
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score,mean_absolute_error, confusion_matrix, silhouette_score
from sklearn.metrics import roc_auc_score,roc_curve, auc, classification_report,precision_score,recall_score,log_loss,f1_score
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier, AdaBoostClassifier
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, LabelEncoder, OneHotEncoder, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer,minmax_scale
from sklearn.svm import SVC, LinearSVC
from sklearn import tree
import pandas_bokeh
from sklearn.decomposition import PCA, KernelPCA
from sklearn.calibration import CalibratedClassifierCV
from numpy import mean, std
import pandas.testing as tm
from scipy import stats
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.tree import ExtraTreeClassifier

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

#other
from math import sqrt
import inspect
from matplotlib.font_manager import FontProperties
from scipy.stats import loguniform, uniform
from bokeh import io

import eli5

from yellowbrick.features import Rank2D
from yellowbrick.features import PCA as PCA_YB
from yellowbrick.features.radviz import RadViz
from yellowbrick.features import pca_decomposition
from yellowbrick.features import Manifold
from yellowbrick.features import JointPlotVisualizer
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.model_selection import LearningCurve
from yellowbrick.model_selection import CVScores
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.features import ParallelCoordinates
from yellowbrick.model_selection import RFECV
from yellowbrick.classifier import ROCAUC
import warnings
warnings.filterwarnings("ignore")

In [2]:
DATASET_URL = "https://gist.githubusercontent.com/YHYeoh/ad1a7f7170c72d621d05a70637540152/raw/5a6059c199e2c46d2f3d258f03d93cfea98e2749/marketing_campaign.csv"
data = pd.read_csv(DATASET_URL, sep = ';')

pd.set_option('plotting.backend','pandas_bokeh')

data.fillna(method = "ffill", inplace = True)
data.isnull().values.any()

label_encoder = LabelEncoder()
enc = OneHotEncoder()

data["Education"] = label_encoder.fit_transform(data["Education"])
print(label_encoder.classes_)
# enc_df = pd.DataFrame(enc.fit_transform(data[["Marital_Status"]]).toarray())
# print(enc.get_feature_names())
# data = data.join(enc_df)
data_copy = data
marital_status_ohe = pd.get_dummies(data["Marital_Status"],prefix="Marital")
ohe_cols = marital_status_ohe.columns
data = pd.concat([data, marital_status_ohe], axis=1)


data['enroll_year'] = pd.DatetimeIndex(data.Dt_Customer).year
data['enroll_month'] = pd.DatetimeIndex(data.Dt_Customer).month
data['enroll_day'] = pd.DatetimeIndex(data.Dt_Customer).day

data.drop(["ID", 'Dt_Customer',"Z_CostContact","Z_Revenue","Marital_Status"], axis=1, inplace=True)

categorical = ['Marital_Status']
numerical = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain', 'enroll_year', 'enroll_month', 'enroll_day']
numerical_no_bool = ['Education','Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth','enroll_day','enroll_month','enroll_year']

y = data.Response
print(y.value_counts())
X = data.drop(['Response'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.3)
X_train_cont = X_train.drop(['AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2','Education','Complain','Year_Birth','Marital_Absurd', 'Marital_Alone', 'Marital_Divorced',
       'Marital_Married', 'Marital_Single', 'Marital_Together',
       'Marital_Widow', 'Marital_YOLO', 'enroll_year', 'enroll_month',
       'enroll_day'],axis=1)
X_test_cont = X_test.drop(['AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2','Education','Complain','Year_Birth','Marital_Absurd', 'Marital_Alone', 'Marital_Divorced',
       'Marital_Married', 'Marital_Single', 'Marital_Together',
       'Marital_Widow', 'Marital_YOLO', 'enroll_year', 'enroll_month',
       'enroll_day'],axis=1)
X_cont_column = X_train_cont.columns
pcaX_train = X_train_cont
pcaX_test = X_test_cont

['2n Cycle' 'Basic' 'Graduation' 'Master' 'PhD']
0    1906
1     334
Name: Response, dtype: int64


In [3]:
def hasmethod(obj, name):
	return inspect.ismethod(getattr(obj, name, None))

def ROC_Curve_Plot(model,X_test,y_test,name):
	predProb = model.predict_proba(X_test)
	preds = predProb[:,1]
	fpr, tpr, threshold = roc_curve(y_test, preds,pos_label=1)
	roc_auc = auc(fpr, tpr)
	plt.close()
	plt.title(name+' Receiver Operating Characteristic')
	plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
	plt.legend(loc = 'lower right')
	plt.plot([0, 1], [0, 1],'r--')
	plt.ylabel('True Positive Rate')
	plt.xlabel('False Positive Rate')
	plt.show()
	return fpr,tpr

def setupPreprocessPipeline(scaler):
	ss = Pipeline(steps=[('scaler',scaler)])
	#ohe = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown = 'ignore'))])
	preprocess = ColumnTransformer(
                    transformers=[
                        ('cont', ss, numerical_no_bool)
                        #('cat', ohe, categorical),
                        #('le', le, ordinal),
                        ],remainder='passthrough')
	return preprocess

def feature_importance(classifier, feature_names, scaler_name):
	if (hasattr(classifier,'coef_')):
		importance = classifier.coef_[0]
	elif (hasattr(classifier,'coefs_')):
		importance = classifier.coefs_
	elif (hasattr(classifier,'feature_importances_')):
		importance = classifier.feature_importances_
	else:
		print("Cannot extract feature importance, skipping")
		return

	#for i,v in enumerate(importance):
	#	print('Feature: %d, Score: %.5f' % (i,v))
	zipped = zip(feature_names, importance)
	df = pd.DataFrame(zipped, columns=["feature", "value"])
	# Sort the features by the absolute value of their coefficient
	df["abs_value"] = df["value"].apply(lambda x: abs(x))
	df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
	df = df.sort_values("abs_value", ascending=False)
	# plot feature importance
	fig, ax = plt.subplots(1, 1, figsize=(16, 9))
	sns.barplot(x="feature",
	            y="value",
	            data=df.head(20),
	           palette=df.head(20)["colors"])
	plt.gcf().subplots_adjust(bottom=0.30)
	ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=14)
	ax.set_title("Top 20 Features for {} w/ {}".format(classifier.__class__.__name__, scaler_name), fontsize=25)
	ax.set_ylabel("Coef", fontsize=22)
	ax.set_xlabel("Feature Name", fontsize=22)
	plt.show()

def evaluation(y, y_hat, title):
	cm = confusion_matrix(y, y_hat)
	precision = precision_score(y, y_hat)
	recall = recall_score(y, y_hat)
	accuracy = accuracy_score(y,y_hat)
	f1 = f1_score(y,y_hat)
	print('Recall: ', recall)
	print('Accuracy: ', accuracy)
	print('Precision: ', precision)
	print('F1: ', f1)
	sns.heatmap(cm,  cmap= 'PuBu', annot=True, fmt='g', annot_kws=    {'size':20})
	plt.xlabel('predicted', fontsize=18)
	plt.ylabel('actual', fontsize=18)
	plt.title(title, fontsize=18)
	plt.show()
    
def metrics_summary(y_test,y_pred):
	tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
	accuracy=accuracy_score(y_test, y_pred)
	precision = precision_score(y_test, y_pred)
	recall =  recall_score(y_test, y_pred) #sensitivity
	specificity = tn / (tn+fp)
	g_mean= sqrt(recall * specificity)
	mse =mean_squared_error(y_test, y_pred, squared=False)
	r2=r2_score(y_test, y_pred)
	ros = roc_auc_score(y_test, y_pred)
	ll = log_loss(y_test, y_pred)
	f1 = f1_score(y_test, y_pred)
	
	metrics_collection_dict ={
        'accuracy':['accuracy',accuracy],
        'precision':['precision',precision],
        'recall':['recall',recall],
        'specificity':['specificity',specificity],
        'g_mean':['g_mean',g_mean],
        'mean_square_error':['mean_square_error',mse],
        'r2':['r2',r2],
        'roc_auc_score':['roc_auc_score',ros],
        'log_loss':['log_loss',ll],
        'f1_score':['f1_score',f1]
    } 
	return metrics_collection_dict

def correlationPlot():
	visualizer = Rank2D(
        features=X.columns, algorithm='covariance'
    )
	visualizer.fit(X, y)                # Fit the data to the visualizer
	visualizer.transform(X)             # Transform the data
	visualizer.show()                   # Finalize and render the figure

def radVisualize():
	visualizer = RadViz() #nice
	visualizer.fit(X, y)
	visualizer.transform(X)
	visualizer.show()
def pca_scatter_plot():
	visualizer = pca_decomposition(X, y, scale=True, classes=['yes','no'])
	visualizer.show()
def precision_recall_curve(model):
	viz = PrecisionRecallCurve(model)
	viz.fit(X_train, y_train)
	viz.score(X_test, y_test)          #ok
	viz.show()
    
def overallClassificationReport(model,classes):
	visualizer = ClassificationReport(model, classes=classes, support=True) #might can change
	visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
	visualizer.score(X_test, y_test)        # Evaluate the model on the test data
	visualizer.show()

def class_predict_err_plot(model,classes):
	visualizer = ClassPredictionError(model, classes=classes)
	# Fit the training data to the visualizer
	visualizer.fit(X_train, y_train)
	# Evaluate the model on the test data
	visualizer.score(X_test, y_test)
	# Draw visualization
	visualizer.show()

def learning_curve_plot(model):
	cv = StratifiedKFold(n_splits=12)
	sizes = np.linspace(0.3, 1.0, 10)

	# Instantiate the classification model and visualizer
	visualizer = LearningCurve(
	    model, cv=cv, scoring='f1_weighted', train_sizes=sizes, n_jobs=4
	)
	visualizer.fit(X, y)        # Fit the data to the visualizer
	visualizer.show()
    
def cv_scores_plot(model):
	cv = StratifiedKFold(n_splits=12, random_state=42, shuffle= True)
	visualizer = CVScores(model, cv=cv, scoring='f1_weighted')

	visualizer.fit(X, y)        # Fit the data to the visualizer
	visualizer.show()
def overall_feature_importance(model):
	labels = list(map(lambda s: s.title(), X.columns))
	viz = FeatureImportances(model, labels=labels, relative=False, topn = 8)
	viz.fit(X, y)
	viz.show()
def RFECV_plot(model):
	cv= StratifiedKFold(5)
	# Instantiate RFECV visualizer with a linear SVM classifier
	visualizer = RFECV(model, cv= cv)
	visualizer.fit(X, y)        # Fit the data to the visualizer
	visualizer.show()      #ok


In [8]:
pcaParamGrid = [{
    'transformer':PCA(),'param_grid':{'n_components':list(range(2,8)),'random_state':[42]
                                      ,'whiten':[True,False]
                                      ,'svd_solver':['full', 'arpack', 'randomized']}
},{
    'transformer':KernelPCA(),
    'param_grid':{'gamma':loguniform(1e-6, 100),'n_components':list(range(2,8))
                  ,'random_state':[42],'kernel':['rbf'],
                  'alpha':loguniform(1e-6, 100),'eigen_solver':['dense', 'arpack']
                  ,'n_jobs':[-1],'max_iter':list(range(1,1000)),'fit_inverse_transform':[True,False]}
},
    {
        'transformer':KernelPCA(),
    'param_grid':{'gamma':loguniform(1e-6, 100),'n_components':list(range(2,8))
                  ,'random_state':[42],'kernel':['poly'],
                  'alpha':loguniform(1e-6, 100),'degree':list(range(3,8))
                  ,'n_jobs':[-1],'max_iter':list(range(1,1000)),'eigen_solver':['dense', 'arpack'],
                 'fit_inverse_transform':[True,False]}
    }
]

In [9]:
    
def scorer(pcamodel, X, y=None):
    try:
        X_val = X.values
    except:
        X_val = X
    
    try:
        y_val = y.values
    except:
        y_val = y
    data_inv = pcamodel.fit(X_val,y_val).transform(X_val)
    reconstructed = pcamodel.inverse_transform(data_inv)
    #find reconstrucition error
    mse = mean_squared_error(reconstructed.ravel(), X_val.ravel())
    return abs(mse)

def pcaComparison(X_train,y_train,X_test,y_test):
    cv=10
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train,y_train)
    X_test_scaled = scaler.transform(X_test)
    pcaPerformance = pd.DataFrame()
    for transformer in pcaParamGrid :
        pca = transformer['transformer']
        param_grid =  transformer['param_grid']
        PCASearch = RandomizedSearchCV(pca,param_grid,n_iter=10,verbose=2,
                         scoring= scorer,
                         n_jobs=-1,cv=cv,random_state=42)
        PCASearch = PCASearch.fit(X_train_scaled,y_train)

        param = PCASearch.best_params_
        name = PCASearch.best_estimator_.__class__.__name__
        score = PCASearch.best_score_
        pcaResult = {"Model":name,"MSE": score,"Parameter":param}
        pcaPerformance = pcaPerformance.append(pcaResult,ignore_index=True)
    pcaPerformance.sort_values(by='MSE',ascending=True,inplace=True)
    return pcaPerformance

In [10]:
pcaResult = pcaComparison(pcaX_train,y_train,pcaX_test,y_test)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [76]:
i=1
for rowNo,result in pcaResult.iterrows():
    mse,model,param = result
    print(str(i)+" "+str(model))
    print(mse)
    print(param)
    print("\n")
    i+=1

1 KernelPCA
3.5866445054136334e-31
{'alpha': 7.577453045410569, 'degree': 7, 'eigen_solver': 'dense', 'fit_inverse_transform': True, 'gamma': 1.2767906371238508e-06, 'kernel': 'poly', 'max_iter': 601, 'n_components': 5, 'n_jobs': -1, 'random_state': 42}


2 KernelPCA
2.5475797815164096e-18
{'alpha': 2.3528990899815337e-06, 'eigen_solver': 'dense', 'fit_inverse_transform': True, 'gamma': 2.3130924416844095e-05, 'kernel': 'rbf', 'max_iter': 167, 'n_components': 3, 'n_jobs': -1, 'random_state': 42}


3 PCA
0.3641436844031753
{'whiten': True, 'svd_solver': 'arpack', 'random_state': 42, 'n_components': 3}




In [37]:
#KernelPCA has the lowest error
kpca = KernelPCA(alpha= 7.577453045410569, degree= 7, eigen_solver= "dense"
                 , fit_inverse_transform= True, gamma= 1.2767906371238508e-06
                 , kernel= "poly", max_iter= 601, n_components= 5, n_jobs= -1, random_state= 42)
scaler= StandardScaler()
X_train_scaled = scaler.fit_transform(pcaX_train,y_train)
X_test_scaled = scaler.transform(pcaX_test)

kpca = kpca.fit(X_train_scaled,y_train)
transformedXTrain = kpca.transform(X_train_scaled)
transformedXTest=kpca.transform(X_test_scaled)


In [82]:
kpcaCol = ["PCA1","PCA2","PCA3","PCA4","PCA5"]
kpcaTrain = pd.DataFrame(transformedXTrain,columns = kpcaCol)
kpcaTest = pd.DataFrame(transformedXTest, columns = kpcaCol)

In [47]:
explained_variance = np.var(transformedXTrain, axis=0)
explained_variance_ratio = explained_variance / np.sum(explained_variance)
print("Explained Variance : " +str(explained_variance))
print("Explained Variance Ratio: "+str(explained_variance_ratio))

Explained Variance : [5.38199521e-05 1.58401800e-05 9.23942910e-06 8.94696808e-06
 7.43816975e-06]
Explained Variance Ratio: [0.5648331  0.16624054 0.09696656 0.09389722 0.07806258]


In [83]:
kpcaTest

,PCA1,PCA2,PCA3,PCA4,PCA5
0,-0.000160,0.008310,0.000826,-0.002955,-0.000923
1,-0.000025,0.006932,0.002545,-0.001326,-0.000602
2,-0.005094,-0.003583,-0.000943,0.001085,-0.002980
3,0.018443,-0.005191,-0.001657,-0.006741,0.000458
4,0.006501,0.008878,0.002508,0.003399,-0.003859
...,...,...,...,...,...
667,0.007894,0.007176,0.000626,-0.001499,0.000149
668,0.007123,0.004609,0.002419,0.001028,0.003577
669,-0.002863,-0.000980,-0.000024,0.002811,-0.001831
670,0.003611,0.004845,-0.000145,0.002129,0.003467


In [34]:
X_train_scaled[]

array([ 0.76100013, -0.85240427, -0.91446557,  1.34095667,  0.97875804,
        0.0482593 ,  1.3826008 , -0.13276687, -0.31339209,  0.23772792,
       -0.66741304,  0.72081595,  1.42453623,  0.70457026, -0.9429703 ])

In [ ]:
models = [
	SGDClassifier(max_iter = 1000,penalty = "elasticnet"),#tol=1e-3),
	LinearSVC(max_iter=2000), 
	GaussianProcessClassifier(),
	ExtraTreeClassifier(),
	BernoulliNB(),
	LogisticRegressionCV(max_iter= 1200), 
	RidgeClassifierCV(),
	SVC(kernel = 'linear',max_iter= -1), 
	Perceptron(),
	PassiveAggressiveClassifier(), 
	DecisionTreeClassifier(), #no coef 
	KNeighborsClassifier(),#no feat_import, use permutation_importance 
	GaussianNB(), #no feat_import, use permutation_importance 
	LGBMClassifier(),#no coef 
	RandomForestClassifier(), #no coef 
	GradientBoostingClassifier(),#no coef 
	PassiveAggressiveClassifier(), 
	ExtraTreesClassifier(), #no coef 
	XGBClassifier(),
	AdaBoostClassifier(), #no coef 
	]

Below column is previous pca attemps, examined by models instead of restructuring mse

In [ ]:

from sklearn.decomposition import KernelPCA
def scorer(pcamodel, X, y=None):

    try:
        X_val = X.values
    except:
        X_val = X
        
    # Calculate and inverse transform the data
    data_inv = pcamodel.fit(X_val,y).transform(X_val)
    data_inv = pcamodel.inverse_transform(data_inv)
    
    # The error calculation
    mse = mean_squared_error(data_inv.ravel(), X_val.ravel())
    
    # Larger values are better for scorers, so take negative value
    return -1.0 * mse

In [ ]:
pcaParamGrid = {{
    'transformer':KernelPCA(),
    'param_grid':{'gamma':[0.0001,0.001, 0.01, 0.05, 0.1, 0.5, 1.0,1.5,2],'n_components':[2, 3, 4,5,6,7,8],'random_state':[42]}
},{
    'transformer':PCA(),'param_grid':{'n_components':[2, 3, 4,5,6,7,8],'random_state':[42]}
}
}